In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')
import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head(2)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN', 'NAME' and 'SPECIAL_CONSIDERATIONS'.
#  Import and read the charity_data.csv.
application_df = application_df.drop(columns=["EIN","NAME", "SPECIAL_CONSIDERATIONS"])
application_df.head(2)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1


In [4]:
# Determine which values to replace if counts are less than ...?
application_counts = application_df.APPLICATION_TYPE.value_counts()

replace_application = list(application_counts[application_counts < 528].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

In [5]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df.CLASSIFICATION.value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(classification_counts[classification_counts < 1883].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

In [6]:
# Generate our categorical variable lists
application_cat = ['APPLICATION_TYPE','AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT']

In [7]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True).drop(columns=application_cat, axis=1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# Split our preprocessed data into our features and target arrays
y = application_df.IS_SUCCESSFUL
X = application_df.drop('IS_SUCCESSFUL', axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78, stratify=y)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

#### Attempt 1 to Optimize the model (accuracy > 75%)

In [11]:
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 60
nodes_hidden_layer2 = 30

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation='relu', input_dim=number_input_features))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation='relu'))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 60)                2520      
                                                                 
 dense_1 (Dense)             (None, 30)                1830      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 4,381
Trainable params: 4,381
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [13]:
# Define the checkpoint path and filenames
os.makedirs("checkpoints1/",exist_ok=True)
checkpoint_path = "checkpoints1/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs.
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

In [14]:
# Train the model
fit_model1 = nn1.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 1s 969us/step - loss: 0.5767 - accuracy: 0.7170
Epoch 2/50
804/804 [==============================] - 1s 920us/step - loss: 0.5569 - accuracy: 0.7271
Epoch 3/50
804/804 [==============================] - 1s 889us/step - loss: 0.5545 - accuracy: 0.7284
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5526 - accuracy: 0.7274
Epoch 5/50
748/804 [==========================>...] - ETA: 0s - loss: 0.5491 - accuracy: 0.7321
Epoch 5: saving model to checkpoints1\weights.05.hdf5
804/804 [==============================] - 1s 904us/step - loss: 0.5507 - accuracy: 0.7309
Epoch 6/50
804/804 [==============================] - 1s 915us/step - loss: 0.5498 - accuracy: 0.7318
Epoch 7/50
804/804 [==============================] - 1s 906us/step - loss: 0.5496 - accuracy: 0.7298
Epoch 8/50
804/804 [==============================] - 1s 905us/step - loss: 0.5477 - accuracy: 0.7319
Epoch 9/50
804/804 [===========================

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5519 - accuracy: 0.7308 - 253ms/epoch - 945us/step
Loss: 0.5519095659255981, Accuracy: 0.7308454513549805


In [16]:
# Save and export your results to an HDF5 file
nn1.save("AlphabetSoupCharity_Optimization_Attempt1.h5")

#### Attempt 2 to Optimize the model (accuracy > 75%)
##### Using a tanh activation instead of sigmoid

In [20]:
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 60
nodes_hidden_layer2 = 30

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation='relu', input_dim=number_input_features))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation='relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation='tanh'))

# Check the structure of the model
nn2.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 60)                2520      
                                                                 
 dense_7 (Dense)             (None, 30)                1830      
                                                                 
 dense_8 (Dense)             (None, 1)                 31        
                                                                 
Total params: 4,381
Trainable params: 4,381
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Compile the model
nn2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define the checkpoint path and filenames
os.makedirs("checkpoints2/",exist_ok=True)
checkpoint_path = "checkpoints2/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs.
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)


# Train the model
fit_model2 = nn2.fit(X_train_scaled, y_train, epochs=50, callbacks=[cp_callback])

Epoch 1/50
804/804 [==============================] - 2s 1ms/step - loss: 0.6352 - accuracy: 0.7014
Epoch 2/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5825 - accuracy: 0.7199
Epoch 3/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5792 - accuracy: 0.7236
Epoch 4/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5755 - accuracy: 0.7258
Epoch 5/50
751/804 [===========================>..] - ETA: 0s - loss: 0.5661 - accuracy: 0.7282
Epoch 5: saving model to checkpoints2\weights.05.hdf5
804/804 [==============================] - 1s 1ms/step - loss: 0.5663 - accuracy: 0.7277
Epoch 6/50
804/804 [==============================] - 1s 943us/step - loss: 0.5602 - accuracy: 0.7292
Epoch 7/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5548 - accuracy: 0.7300
Epoch 8/50
804/804 [==============================] - 1s 1ms/step - loss: 0.5576 - accuracy: 0.7308
Epoch 9/50
804/804 [==============================] - 1s 1m

In [22]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5596 - accuracy: 0.7301 - 241ms/epoch - 900us/step
Loss: 0.5596135854721069, Accuracy: 0.7301457524299622


In [23]:
# Save and export your results to an HDF5 file
nn2.save("AlphabetSoupCharity_Optimization_Attempt2.h5")

#### Attempt 3 to Optimize the model (accuracy > 75%)
##### Doubling the number of iterations

In [29]:
number_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 = 60
nodes_hidden_layer2 = 30

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation='relu', input_dim=number_input_features))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation='relu'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation='tanh'))

# Check the structure of the model
nn3.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 60)                2520      
                                                                 
 dense_13 (Dense)            (None, 30)                1830      
                                                                 
 dense_14 (Dense)            (None, 1)                 31        
                                                                 
Total params: 4,381
Trainable params: 4,381
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model
nn3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Define the checkpoint path and filenames
os.makedirs("checkpoints3/",exist_ok=True)
checkpoint_path = "checkpoints3/weights.{epoch:02d}.hdf5"

# Create a callback that saves the model's weights every 5 epochs.
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch',
    period=5)

# Train the model
fit_model3 = nn3.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 940us/step - loss: 0.8056 - accuracy: 0.6939
Epoch 2/100
804/804 [==============================] - 1s 877us/step - loss: 0.6336 - accuracy: 0.7097
Epoch 3/100
804/804 [==============================] - 1s 862us/step - loss: 0.5999 - accuracy: 0.7182
Epoch 4/100
804/804 [==============================] - 1s 934us/step - loss: 0.6175 - accuracy: 0.7114
Epoch 5/100
800/804 [============================>.] - ETA: 0s - loss: 0.5999 - accuracy: 0.7192
Epoch 5: saving model to checkpoints3\weights.05.hdf5
804/804 [==============================] - 1s 964us/step - loss: 0.6001 - accuracy: 0.7187
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5980 - accuracy: 0.7190
Epoch 7/100
804/804 [==============================] - 1s 927us/step - loss: 0.5883 - accuracy: 0.7270
Epoch 8/100
804/804 [==============================] - 1s 742us/step - loss: 0.6033 - accuracy: 0.7150
Epoch 9/100
804/804 [==================

In [32]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5918 - accuracy: 0.7312 - 218ms/epoch - 814us/step
Loss: 0.5918353796005249, Accuracy: 0.731195330619812


In [31]:
# Save and export your results to an HDF5 file
nn3.save("AlphabetSoupCharity_Optimization_Attempt3.h5")